# Assignment 2: Credit Card Spending Analysis & Reduction Recommendations

**Objective:** Analyze credit card statement and recommend expense reductions based on user-specified percentage

**Process:**
1. Extract statement summary and transactions from image
2. Categorize expenses using AI
3. Get user's target reduction percentage
4. Generate smart recommendations for which categories to reduce
5. Display detailed breakdown with savings

## Step 1: Import Required Libraries

In [12]:
import os
import base64
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List

## Step 2: Initialize AI Model and Load Image

In [13]:
# Load environment variables
load_dotenv()

# Initialize OpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3,  # Slightly creative for better recommendations
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Function to encode image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Load the credit card statement image
image_path = "sample_statements/statement3.png"  # Change this to your statement
base64_image = encode_image(image_path)

print(f"[SUCCESS] Model initialized and image loaded: {image_path}")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Step 3: Extract Statement Summary and Transactions

In [ ]:
# Define data structure for extraction
class Transaction(BaseModel):
    description: str = Field(description="Transaction description")
    amount: float = Field(description="Transaction amount in INR (positive for debits, negative for credits)")
    category: str = Field(description="Expense category: Cash Withdrawal, Shopping, Dining, Bills, Transfer, Interest, or Other")

class StatementAnalysis(BaseModel):
    total_debits: float = Field(description="Total debit amount from statement summary")
    total_credits: float = Field(description="Total credit amount from statement summary")
    closing_balance: float = Field(description="Closing balance from statement")
    transactions: List[Transaction] = Field(description="List of all transactions with categories")

# Create parser
parser = JsonOutputParser(pydantic_object=StatementAnalysis)

# Create extraction prompt
extraction_prompt = f"""You are a credit card statement analysis expert.

Analyze this credit card statement image and extract:

1. Statement Summary:
   - Total Debits (total spending)
   - Total Credits (payments/refunds received)
   - Closing Balance

2. All Individual Transactions:
   - Description
   - Amount (use positive for debits/spending, negative for credits)
   - Category (categorize each transaction into: Cash Withdrawal, Shopping, Dining, Bills, Transfer, Interest, or Other)

Analyze the transaction descriptions carefully and assign appropriate categories.

Return data in this JSON format:
{parser.get_format_instructions()}

Be precise and extract all visible transactions.
"""

# Send to AI
message = HumanMessage(
    content=[
        {"type": "text", "text": extraction_prompt},
        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
    ]
)

print("[INFO] Extracting statement data and categorizing transactions...\n")
response = llm.invoke([message])
statement_data = parser.parse(response.content)

print("[SUCCESS] Data extracted successfully!")
print(f"\nTotal Debits: INR {statement_data['total_debits']:,.2f}")
print(f"Total Credits: INR {statement_data['total_credits']:,.2f}")
print(f"Closing Balance: INR {statement_data['closing_balance']:,.2f}")
print(f"Total Transactions: {len(statement_data['transactions'])}")

## Step 4: Analyze Spending by Category

In [ ]:
# Calculate category-wise spending
category_spending = {}

for txn in statement_data['transactions']:
    category = txn['category']
    amount = txn['amount']
    
    # Only count debits (positive amounts) as spending
    if amount > 0:
        if category not in category_spending:
            category_spending[category] = 0
        category_spending[category] += amount

# Sort by spending amount (highest first)
sorted_categories = sorted(category_spending.items(), key=lambda x: x[1], reverse=True)

# Display category breakdown
print("="*60)
print("SPENDING BREAKDOWN BY CATEGORY")
print("="*60)
for category, amount in sorted_categories:
    percentage = (amount / statement_data['total_debits']) * 100
    print(f"{category:20s}: INR {amount:>12,.2f} ({percentage:>5.1f}%)")
print("="*60)
print(f"{'TOTAL SPENDING':20s}: INR {statement_data['total_debits']:>12,.2f}")
print("="*60)

## Step 5: Get User Input for Target Reduction

In [ ]:
# Get user input for reduction percentage
reduction_percentage = float(input("Enter target reduction percentage (e.g., 20 for 20%): "))

# Calculate target amounts
current_spending = statement_data['total_debits']
target_reduction_amount = current_spending * (reduction_percentage / 100)
target_spending = current_spending - target_reduction_amount

print(f"\n[INFO] Target: Reduce spending by {reduction_percentage}%")
print(f"Current Spending: INR {current_spending:,.2f}")
print(f"Target Spending: INR {target_spending:,.2f}")
print(f"Amount to Save: INR {target_reduction_amount:,.2f}")

## Step 6: Generate AI-Powered Reduction Recommendations (FIXED VERSION)

In [ ]:
# Prepare data for AI analysis
analysis_context = f"""Credit Card Spending Analysis:

Current Total Spending: INR {current_spending:,.2f}
Target Reduction: {reduction_percentage}%
Amount to Save: INR {target_reduction_amount:,.2f}

Category-wise Spending:
"""

for category, amount in sorted_categories:
    percentage = (amount / current_spending) * 100
    analysis_context += f"- {category}: INR {amount:,.2f} ({percentage:.1f}%)\n"

# Create recommendation prompt
recommendation_prompt = f"""{analysis_context}

Based on this spending analysis, provide smart recommendations on which expense categories should be reduced and by how much to achieve the target reduction of {reduction_percentage}%.

Guidelines:
1. Prioritize reducing non-essential categories (Cash Withdrawal, Shopping, Dining)
2. Be conservative with essential categories (Bills, Interest)
3. Provide specific reduction percentages for each category
4. Ensure total savings equals INR {target_reduction_amount:,.2f}
5. Provide practical, actionable advice

Return recommendations in JSON format:
{{
  "recommendations": [
    {{
      "category": "category name",
      "current_spending": amount,
      "reduction_percentage": percentage,
      "amount_to_save": amount,
      "new_spending": amount,
      "advice": "specific actionable advice"
    }}
  ],
  "total_savings": {target_reduction_amount}
}}
"""

print("[INFO] Generating AI-powered recommendations...\n")
recommendation_response = llm.invoke([HumanMessage(content=recommendation_prompt)])

# ===== JSON PARSING FIX =====
# Extract JSON from response (AI might wrap it in markdown code blocks)
response_text = recommendation_response.content

# Check if response is wrapped in markdown code blocks
if "```json" in response_text:
    # Extract JSON from ```json ... ``` blocks
    json_start = response_text.find("```json") + 7
    json_end = response_text.find("```", json_start)
    response_text = response_text[json_start:json_end].strip()
elif "```" in response_text:
    # Extract JSON from ``` ... ``` blocks
    json_start = response_text.find("```") + 3
    json_end = response_text.find("```", json_start)
    response_text = response_text[json_start:json_end].strip()

# Parse recommendations with error handling
try:
    recommendations = json.loads(response_text)
    print("[SUCCESS] Recommendations generated!")
except json.JSONDecodeError as e:
    print(f"[ERROR] Failed to parse JSON: {e}")
    print(f"\nRaw AI response:\n{recommendation_response.content}")
    raise

## Step 7: Display Detailed Recommendations

In [ ]:
# Display comprehensive recommendations
print("\n" + "="*70)
print("SPENDING REDUCTION RECOMMENDATIONS")
print("="*70)
print(f"\nCurrent Total Spending: INR {current_spending:,.2f}")
print(f"Target Reduction: {reduction_percentage}%")
print(f"Target Total Spending: INR {target_spending:,.2f}")
print(f"Amount to Save: INR {target_reduction_amount:,.2f}")
print("\n" + "-"*70)
print("CATEGORY-WISE REDUCTION PLAN")
print("-"*70)

for rec in recommendations['recommendations']:
    print(f"\n{rec['category'].upper()}")
    print(f"  Current Spending:     INR {rec['current_spending']:>12,.2f}")
    print(f"  Reduce by:            {rec['reduction_percentage']:>12.0f}%")
    print(f"  Amount to Save:       INR {rec['amount_to_save']:>12,.2f}")
    print(f"  New Target Spending:  INR {rec['new_spending']:>12,.2f}")
    print(f"  Advice: {rec['advice']}")

print("\n" + "="*70)
print(f"TOTAL PROJECTED SAVINGS: INR {recommendations['total_savings']:,.2f}")
print("="*70)
print("\n[SUCCESS] Analysis complete! Follow these recommendations to reduce your")
print(f"credit card spending by {reduction_percentage}% and save INR {target_reduction_amount:,.2f}")
print("="*70)